In [1]:
def my_decorator(func):
    def wrapper():
        print("Something before the function is called.")
        func()
        print("Something after the function is called.")
    return wrapper

@my_decorator
def my_function():
    print("My function is called.")

my_function()


Something before the function is called.
My function is called.
Something after the function is called.


In [2]:
def uppercase_decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def exclamation_decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result + "!"
    return wrapper

@exclamation_decorator
@uppercase_decorator
def greet(name):
    return f"Hello, {name}"

print(greet("Alice"))


HELLO, ALICE!


In [3]:
import requests
from bs4 import BeautifulSoup

# Decorator for crawling IMDb movie details
def crawl_imdb_movie_details(func):
    def wrapper(movie_url):
        # Check if the URL is from IMDb
        if 'imdb.com/title/' not in movie_url:
            print("Invalid IMDb URL")
            return

        # Send an HTTP GET request to the IMDb movie page
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(movie_url, headers=headers)

        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract movie details (you can customize this part)
            title = soup.find('div', {'class': 'sc-afe43def-3 EpHJp'}).text.replace('Original title:', '').strip()
            rating = soup.find('span', {'class': 'sc-bde20123-1 iZlgcd'}).text.strip()
            description = soup.find('span', {'class': 'sc-466bb6c-0 kJJttH'}).text.strip()
            
            print(f"Title: {title}")
            print(f"Rating: {rating}")
            print(f"Description: {description}")

            # Call the original function
            return func(movie_url)

        except:
            
            print(f"Failed to fetch data from IMDb URL: {movie_url}")

    return wrapper

# Example usage of the decorator
@crawl_imdb_movie_details
def get_imdb_movie_data(movie_url):
    # Here, you can define additional processing for the IMDb data
    print(f"Crawling additional data for IMDb URL: {movie_url}")

# Example IMDb movie URL
movie_url = 'https://www.imdb.com/title/tt0111161/'  # The Shawshank Redemption

# Use the decorated function to crawl IMDb data
get_imdb_movie_data(movie_url)


Title: The Shawshank Redemption
Rating: 9.3
Description: Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.
Crawling additional data for IMDb URL: https://www.imdb.com/title/tt0111161/


In [4]:
import requests
from bs4 import BeautifulSoup

def crawl_movie_page_decorator(crawler_method):
    def wrapper(self, movie_url):
        if movie_url in self.visited_urls:
            return

        # Send an HTTP GET request to the IMDb movie page
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(movie_url, headers=headers)
        
        try:
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Extract movie details (you can customize this part)
                title = soup.find('div', {'class': 'sc-afe43def-3 EpHJp'}).text.replace('Original title:', '').strip()
                rating = soup.find('span', {'class': 'sc-bde20123-1 iZlgcd'}).text.strip()
                description = soup.find('span', {'class': 'sc-466bb6c-0 kJJttH'}).text.strip()
                
                print(f"Title: {title}")
                print(f"Rating: {rating}")
                print(f"Description: {description}")
                
                # Add visited url to visited_urls
                self.visited_urls.add(movie_url)
                
        except Exception as e:
            print(f"Error crawling {movie_url}: {str(e)}")

    return wrapper

class IMDBCrawler:
    def __init__(self):
        self.base_url = "https://www.imdb.com"
        self.visited_urls = set()

    @crawl_movie_page_decorator
    def crawl_movie_page(self, movie_url):
        pass  # The decorator adds functionality to this method

    def crawl_top_rated_movies(self, max_movies=10):
        top_rated_url = f"{self.base_url}/chart/top"
        try:
            response = requests.get(top_rated_url)
            # Send an HTTP GET request to the IMDb movie page
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(top_rated_url, headers=headers)
        
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                movie_links = soup.find_all("a", {"class": "ipc-title-link-wrapper"})
                
                for link in movie_links[:max_movies]:
                    print('-----------------------------------------------------------')
                    movie_url = self.base_url + link['href']
                    self.crawl_movie_page(movie_url)
                    print('Rank:', movie_url[-1])
                    print('URL:', movie_url)
                    
        except Exception as e:
            print(f"Error crawling top-rated movies: {str(e)}")

if __name__ == "__main__":
    imdb_crawler = IMDBCrawler()
    imdb_crawler.crawl_top_rated_movies(max_movies=5)


-----------------------------------------------------------
Title: The Shawshank Redemption
Rating: 9.3
Description: Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.
Rank: 1
URL: https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1
-----------------------------------------------------------
Title: The Godfather
Rating: 9.2
Description: Don Vito Corleone, head of a mafia family, decides to hand over his empire to his youngest son Michael. However, his decision unintentionally puts the lives of his loved ones in grave danger... Read all
Rank: 2
URL: https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2
-----------------------------------------------------------
Title: The Dark Knight
Rating: 9.0
Description: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.
Rank